In [ ]:
import os
import glob
import textgrid
import sys
import pandas as pd
from re import match, sub, findall, finditer

path           = './'
word_level     = path + 'word_level'
video          = path + 'video' 
audio          = path +'audio' 
documents      = path + 'documents', 
metadata       = path+ 'metadata', 
transcriptions = path +'transcriptions_annotations'
root_dir = '../'
word_level_timing = root_dir+'word_level_timings'
motion_label = root_dir+'motion_labels'

In [ ]:
def get_all_textgrid_files(path):
    filenames = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".TextGrid"):
                filenames.append(os.path.join(root, file))
    return filenames
word_level_timing_annotation = get_all_textgrid_files(word_level_timing)
print(word_level_timing_annotation[0])      

In [ ]:
def clean_utt(utt, literal=False):
    if not literal:
        #replace variants, partial and misspoken words with standard spelling
        utt = sub("""<[vpm]="(.+?)">.+?</[vpm]>""", lambda m:m.group(1), utt)
        #remove fillers like "{F aehm}" entirely
        utt = sub("""{.*?}""", "", utt)
        
        #TO DO: resolve complex replacements like "(der + der) + die) Katze"
        
    else:
        #remove brackets from fillers, i.e. "{F aehm}" becomes "aehm"
        utt = sub("""{(.*?)}""",lambda m:m.group(1),utt)
    #remove all remaining xml-style tags    
    utt = sub("""<.*?>""","",utt)
    #remove open tags at the end of an utterance (can be removed once problems with the TextGrids are fixed)
    utt = sub("""<.*$""","",utt)
    #remove all remaining punctuation and brackets
    utt = sub("""[\.:;,\(\)\+\$]""","",utt)
    #remove whitespace at the beginning and end of an utterance
    utt = utt.strip()
    #replace any amount of whitespace with a single space
    utt = sub("""\s+"""," ",utt)
    return utt


In [ ]:
for files in word_level_timing_annotation:
    for index , tg in enumerate(textgrid.TextGrid.fromFile(files)):
        for interval in tg:
            print(clean_utt(interval.mark))